### 지원사업공고 update (추가, 수정, 삭제)

In [14]:
import requests
import json

from bs4 import BeautifulSoup
from tqdm import tqdm
from datetime import datetime, timedelta

In [15]:
# 기존에 저장되어있는 지원사업 공고 파일 및 지난 공고를 불러옴
with open('bizinfo.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

with open('bizold.json', 'r', encoding='utf-8') as f:
    old_infos = json.load(f)

In [16]:
for _, value in data.items():
    print(f'{len(value)}')

for _, value in old_infos.items():
    print(len(value))

155
325
82
187
117
119
389
18
0
0
0
0
0
0
0
0


In [17]:
# 사이트 접속 차단을 받지 않기 위한 header 지정
header = {
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36'
}

# 기업마당의 해시코드를 저장해놓은 dictionary
hashcode_dict = {'01':'금융',
                '02':'기술',
                '03':'인력',
                '07':'수출',
                '08':'내수',
                '09':'창업',
                '10':'경영',
                '12':'기타'}

# key-value를 반대로 저장
reverse_dict = {value: key for key, value in hashcode_dict.items()}

# 결과 출력
print(reverse_dict)

{'금융': '01', '기술': '02', '인력': '03', '수출': '07', '내수': '08', '창업': '09', '경영': '10', '기타': '12'}


In [18]:
def download_file(URL, header=header):
    '''지원사업 공고에 있는 파일을 다운로드하는 함수'''
    # bs4로 해당 URL의 html 구조 파싱
    response = requests.get(URL, headers=header)
    soup = BeautifulSoup(response.text, 'html.parser')

    # 파일의 정보를 담을 빈 딕셔너리 선언
    file_info = {}

    # 다운로드 파일 링크를 담을 빈 리스트 / 파일명을 담을 빈 문자열 선언
    down_link, down_title = [], ''

    # '본문출력파일'의 파일만 다운로드 하도록 수행하는 반복문 + 조건문
    for idx in range(len(soup.select('div.attached_file_list h3'))):
        if soup.select('div.attached_file_list h3')[idx].text == '본문출력파일':
            down_link = soup.select('div.attached_file_list div.right_btn')[-1].select('a')
            file_path = soup.select_one('div.category span').text.strip()

            # 다운 받을 공고문 파일의 이름을 공고문의 title로 바꿔서 정리
            file_title = soup.select_one('h2.title').text.strip()
            down_title = file_title + '.' + soup.select('div.attached_file_list div.file_name')[-1].text.strip().split('.')[-1]
        else:
            continue

    # 다운로드 파일링크는 내부 url이기 때문에 앞에 도메인 주소를 붙여줘야 함.
    base_url = 'https://www.bizinfo.go.kr'
    down_url = base_url + down_link[1].attrs['href']
    

    # 다운로드된 파일을 저장할 파일 경로
    save_path = f'C:/Users/every/Desktop/Project/team_project/SeSAC_Final/AI-for-Creating-Business-Plan-Drafts/{file_path}/{down_title}'

    # HTTP 요청을 보내 파일 다운로드
    response = requests.get(down_url, stream=True)
    try:
        with open(save_path, "wb") as file:
            for chunk in response.iter_content(1024):  # 1024바이트씩 저장
                file.write(chunk)
    except:
        print(f"파일 다운로드 실패. 상태 코드: {response.status_code}")
        print(f"다운로드 실패한 공고 명 : {down_title}")


    # 파일의 정보를 딕셔너리 형태로 담음
    # 프레임으로 관리 및 공고 수정 사항을 빠르게 탐색하기 위한 정보 모음집
    file_info['지원사업 공고명'] = file_title
    file_info['소관부처·지자체'] = soup.select('div.view_cont li div.txt')[0].text.strip()
    file_info['사업수행기관'] = soup.select('div.view_cont li div.txt')[1].text.strip()
    file_info['신청기간'] = soup.select('div.view_cont li div.txt')[2].text.strip().replace("\r", " ").replace("\n", "").replace("\t", "")
    file_info['공고파일명'] = file_path + '/' + down_title

    return file_info, file_path

def fetch_links(hashcode, header=header):
    '''기업마당의 메인에 있는 공고 각각의 URL을 리스트로 반환'''

    # 기업마당의 공고 URL은 BASE_URL + td class='txt_l a'의 형식으로 되어 있음
    BASE_URL = 'https://www.bizinfo.go.kr/web/lay1/bbs/S1T122C128/AS/74/'
    idx = 1         # 페이지를 넘기기 위한 인덱스
    links = []      # 링크를 정리해둘 리스트 선언

    # 공고가 없는 페이지가 나올 때까지 반복문
    while True:
        URL = f'https://www.bizinfo.go.kr/web/lay1/bbs/S1T122C128/AS/74/list.do?hashCode={hashcode}&cpage={idx}'
        response = requests.get(URL, headers=header)
        soup = BeautifulSoup(response.text, 'html.parser')

        # 공고가 없는 페이지가 나올 시 반복문 탈출
        if len(soup.select('div.sub_cont tr')) == 2:
            break
        
        # links에 URL 저장
        # 해당 페이지에 있는 전체 URL에 대하여 실행
        links += [BASE_URL + link.attrs['href'] for link in soup.select('td.txt_l a')]

        # 페이지 넘김
        idx += 1

    return links

def update_fetch_links(hashcode, header=header):
    '''기업마당의 메인에 있는 공고 각각의 URL을 리스트로 반환'''

    # 기업마당의 공고 URL은 BASE_URL + td class='txt_l a'의 형식으로 되어 있음
    BASE_URL = 'https://www.bizinfo.go.kr/web/lay1/bbs/S1T122C128/AS/74/'
    idx = 1         # 페이지를 넘기기 위한 인덱스
    links, titles = [], []     # 링크를 정리해둘 리스트 선언

    # 공고가 없는 페이지가 나올 때까지 반복문
    while True:
        URL = f'https://www.bizinfo.go.kr/web/lay1/bbs/S1T122C128/AS/74/list.do?hashCode={hashcode}&cpage={idx}'
        response = requests.get(URL, headers=header)
        soup = BeautifulSoup(response.text, 'html.parser')

        # 공고가 없는 페이지가 나올 시 반복문 탈출
        if len(soup.select('div.sub_cont tr')) == 2:
            break
        
        # links에 URL 저장
        # 해당 페이지에 있는 전체 URL에 대하여 실행
        links += [BASE_URL + link.attrs['href'] for link in soup.select('td.txt_l a')]
        titles += [title.text.strip() for title in soup.select('td.txt_l a')]

        # 페이지 넘김
        idx += 1

    return links, titles

In [19]:
# 오늘 날짜 가져오기
today = datetime.today().date()        # 현재 날짜 (YYYY-MM-DD) # 실험상 임의로 오늘의 날짜를 1일 뒤로 지정

# 마감기한이 끝났거나 혹은 기한이 저장되어 있지 않은 공고의 삭제
for key, value in tqdm(list(data.items())):  # 딕셔너리 항목을 리스트로 변환하여 순회 (수정 가능하게)

    links, titles = update_fetch_links(reverse_dict[key])  # 기존의 DB와 비교할 오늘의 공고 목록
    now_titles = [] # 현재 DB에 있는 항목을 저장할 리스트

    expired_items = []  # 삭제할 항목을 저장할 리스트
    
    for idx in value[:]:  # 원본 리스트를 복사하여 순회

        now_titles.append(idx['지원사업 공고명']) # 현재 공고의 타이틀을 저장

        if idx['신청기간'][0] == '2':  # 마감 기한이 숫자로 시작하는 경우
            deadline = datetime.strptime(idx['신청기간'][-10:], "%Y.%m.%d").date()
            if deadline < today:  # 마감 기한이 현재 날짜보다 이전이라면
                old_infos[key].append(idx)  # 기존 정보 저장
                expired_items.append(idx)  # 삭제할 항목 저장
    

        else:  # 마감 기한이 날짜가 아닌 경우 (예산 소진시까지, 추후 공지, 상시 접수 등)
            if idx['지원사업 공고명'] not in titles:
                old_infos[key].append(idx)  # 기존 정보 저장
                expired_items.append(idx)  # 삭제할 항목 저장

        
    # 만료된 항목을 기존 리스트에서 제거
    for item in expired_items:
        value.remove(item)
    
    # 만약 해당 key의 리스트가 비었다면, 아예 삭제
    if not value:
        del data[key]

    # 새로운 항목에 대하여 기존 딕셔너리에 저장
    for i in range(len(titles)):
        if titles[i] not in now_titles:
            print(titles[i])
            file_info, file_path = download_file(links[i])
            data[file_path].append(file_info)

  0%|          | 0/8 [00:00<?, ?it/s]

[서울] 콘텐츠 기업의 투자유치 활성화를 위한(투자 네크워킹 데이) 리버스 피칭 참여기업 모집 공고
2025년 수산발전기금 사업희망자(김 업체) 추가 모집 공고
2025년 1-2차 ICT벤처 IR 참가기업 모집 공고
2025년 1차 유동화회사보증 지원 공고


 12%|█▎        | 1/8 [00:16<01:52, 16.13s/it]

2025년 클라우드 종합솔루션 지원사업 참여기업 모집 공고(제조분야)
2025년 경상남도 디지털 혁신거점 조성 지원사업 디지털 혁신지원(R&BD) 지원 통합 공고(디지털기업in경남)
2025년 글로벌기업산업기술연계R&D 사업 신규지원 대상과제 공고
2025년 KRISS Q-FABㆍQ-LAB 연계활용 양자전환지원사업 공모
2025년 스마트시티 혁신기술 발굴 및 혁신서비스 모델 검증 사업 공모
2025년 KRISS 양자전환 애로기술 해결 사업 공모
2025년 로봇융합 비즈니스 지원사업 지원기업 모집 수정 공고
[부산] 2025년 중소기업 ISO 인증 지원사업 모집 공고
[부산] 2025년 중소기업 기술자료 보호 지원사업 모집 공고
[대전] 2025년 첨단소재 및 반도체산업 시제품 제작 지원사업 지원기업 모집 공고
2025년 전략산업 3D프린팅 기술 부품 실증 지원 모집 공고(대전형 3D프린팅 산업 활성화 지원사업)
[대전] 2025년 3D프린팅 후가공 기업 공정개선 지원 모집 공고(대전형 3D프린팅 산업 활성화 지원사업)
[대전] 2025년 3D프린팅 기술기반 전략산업 고도화 지원 모집 공고(대전형 3D프린팅 산업 활성화 지원사업)
[대구] 2025년 소공인 제품ㆍ기술 경쟁력 향상 지원사업 모집 공고
2025년 원자력 강소기업 성장 견인 지원사업 모집 공고
[경북] 포항시 2025년 수소연료전지 경쟁력 강화사업 수혜기업 모집 공고


 25%|██▌       | 2/8 [00:56<03:02, 30.49s/it]

...

 75%|███████▌  | 6/8 [02:02<00:35, 17.90s/it]

[인천] 2025년 청년고용 우수기업 근로환경 개선사업 참여기업 모집 공고
[경기] 2025년 ICT 제조창업 시제품 제작 및 제품화 지원 모집 공고
2025년 지역특화 제조데이터 활성화사업 수행기관 모집 공고
[충북] 2025년 디지털무역종합지원센터(deXter) 활용 화상상담회 참여기업 모집 공고
[부산] 2025년 지능정보기술 융합기업 디자인혁신 지원사업 지원과제 모집 공고
[부산] 2025년 중소기업 디자인개발 지원 사업 지원과제 모집 공고
[경기] 2025년 사회적경제 성장패키지 참여기업 모집 공고
[전북] 전주시 2025년 농식품기업 역량강화 패키지 지원사업 공고
[부산] 2025년 리사이클 디자인 산업 육성 사업 모집 공고
[세종] 2025년 중소기업 ESG 경영지원 사업 참여기업 모집 공고
2025년 전남 문화콘텐츠산업 지원사업 모집 재공고
[서울] 2025년 중소기업 워라밸 포인트제 참여기업 모집 공고
2025년 협력사 ESG 지원사업 참여기업 모집 공고
[부산] 2025 중소기업 공영홈쇼핑 방송판매(지역 특화제품 판로) 지원사업 참가기업 모집 공고
2025년 녹색기후기금(GCF) 사업개발 및 개도국 지원사업 발굴사업 모집 공고
[충북] 2025년 기업성장지원프로그램 모집 공고
2025년 세부지원사업 지원기업 모집 공고(K-방산 생태계 활성화 및 강소기업 맞춤형 패키지 지원사업)
[경북] 포항시 2025년 농산업 현장 맞춤형 제품고도화 사업 수혜기업 모집 공고
[전남] 2025년 JCEP 마케팅 지원사업 공고(전남콘텐츠기업지원센터 운영사업)


 88%|████████▊ | 7/8 [02:55<00:29, 29.29s/it]

2025년 제1차 윈윈 아너스(WIN-WIN HONORS) 프로젝트 추진계획 공고
2025년 상생컨소시엄 기획사업 모집 공고


100%|██████████| 8/8 [03:00<00:00, 22.57s/it]


In [22]:
for _, value in data.items():
    print(len(value))

for _, value in old_infos.items():
    print(len(value))

158
315
79
172
105
110
368
20
1
26
6
29
16
13
40
0


In [21]:
with open('bizinfo.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, indent=4, ensure_ascii=False)

with open('bizold.json', 'w', encoding='utf-8') as f:
    json.dump(old_infos, f, indent=4, ensure_ascii=False)